<a href="https://colab.research.google.com/github/kyledang14-web/MIS-Python-Practice./blob/main/PromptEngineering1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import json
import re

def classify_and_respond(customer_message):
    """
    GoldenGate Gadgets support classifier and responder.
    Returns JSON with intent, clarify, and answer fields.
    """
    msg_lower = customer_message.lower()

    # Intent classification
    intent = None
    if any(word in msg_lower for word in ['refund', 'return', 'money back', 'cancel order']):
        intent = 'refund'
    elif any(word in msg_lower for word in ['shipping', 'delivery', 'tracking', 'where is', 'arrived', 'receive']):
        intent = 'shipping'
    elif any(word in msg_lower for word in ['broken', 'not working', 'won\'t', 'wont', 'doesn\'t', 'doesnt',
                                              'error', 'issue', 'problem', 'tech', 'support', 'fix', 'turn on']):
        intent = 'tech'
    else:
        # Default to tech for ambiguous cases
        intent = 'tech'

    # Check for order_id (common patterns: #12345, order 12345, etc.)
    order_id_match = re.search(r'(?:#|order\s*|id\s*)[:\s]*([A-Z0-9]{5,})', customer_message, re.IGNORECASE)
    has_order_id = order_id_match is not None

    # Check for symptom description (for tech issues)
    tech_symptoms = ['won\'t', 'wont', 'doesn\'t', 'doesnt', 'not', 'broken', 'error', 'problem']
    has_symptom = any(symptom in msg_lower for symptom in tech_symptoms) and len(customer_message.split()) > 5

    # Determine if clarification is needed
    clarify = "NONE"
    answer = ""

    if intent in ['refund', 'shipping']:
        if not has_order_id:
            clarify = "What's your order ID?"
            answer = "PENDING_INFO"
        else:
            if intent == 'refund':
                answer = "I've initiated your refund request. You'll receive a confirmation email within 24 hours with the return shipping label and expected refund timeline."
            else:  # shipping
                answer = "I've located your order and it's currently in transit. You can track it in real-time at goldengate.com/track using your order ID."
    else:  # tech
        if not has_symptom:
            clarify = "What exactly is the device doing (or not doing)?"
            answer = "PENDING_INFO"
        else:
            answer = "Let's troubleshoot this issue together. Please try a hard reset by holding the power button for 15 seconds, then let me know if that resolves the problem."

    return json.dumps({
        "intent": intent,
        "clarify": clarify,
        "answer": answer
    }, indent=2)


# Demo examples
print("=" * 60)
print("DEMO 1: Refund request with order ID")
print("=" * 60)
customer_message = "I need a refund for order #A7X92"
print(f"Customer: {customer_message}\n")
print(classify_and_respond(customer_message))

print("\n" + "=" * 60)
print("DEMO 2: Refund request WITHOUT order ID")
print("=" * 60)
customer_message = "I want to return my gadget"
print(f"Customer: {customer_message}\n")
print(classify_and_respond(customer_message))

print("\n" + "=" * 60)
print("DEMO 3: Shipping inquiry with order ID")
print("=" * 60)
customer_message = "Where is my package? Order ID: BG4521"
print(f"Customer: {customer_message}\n")
print(classify_and_respond(customer_message))

print("\n" + "=" * 60)
print("DEMO 4: Tech issue with symptom")
print("=" * 60)
customer_message = "My device won't turn on at all"
print(f"Customer: {customer_message}\n")
print(classify_and_respond(customer_message))

print("\n" + "=" * 60)
print("DEMO 5: Tech issue WITHOUT symptom")
print("=" * 60)
customer_message = "I need tech support"
print(f"Customer: {customer_message}\n")
print(classify_and_respond(customer_message))

DEMO 1: Refund request with order ID
Customer: I need a refund for order #A7X92

{
  "intent": "refund",
  "clarify": "NONE",
  "answer": "I've initiated your refund request. You'll receive a confirmation email within 24 hours with the return shipping label and expected refund timeline."
}

DEMO 2: Refund request WITHOUT order ID
Customer: I want to return my gadget

{
  "intent": "refund",
  "clarify": "What's your order ID?",
  "answer": "PENDING_INFO"
}

DEMO 3: Shipping inquiry with order ID
Customer: Where is my package? Order ID: BG4521

{
  "intent": "shipping",
  "clarify": "NONE",
  "answer": "I've located your order and it's currently in transit. You can track it in real-time at goldengate.com/track using your order ID."
}

DEMO 4: Tech issue with symptom
Customer: My device won't turn on at all

{
  "intent": "tech",
  "clarify": "NONE",
  "answer": "Let's troubleshoot this issue together. Please try a hard reset by holding the power button for 15 seconds, then let me know i